In [21]:
import pandas as pd
import numpy as np
import itertools
from random import shuffle

In [22]:
# read in original data as a dataframe
df = pd.read_csv('adult.data', header=None, skipinitialspace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [23]:
df.info()
df_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship",
            "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32561 non-null  int64 
 1   1       32561 non-null  object
 2   2       32561 non-null  int64 
 3   3       32561 non-null  object
 4   4       32561 non-null  int64 
 5   5       32561 non-null  object
 6   6       32561 non-null  object
 7   7       32561 non-null  object
 8   8       32561 non-null  object
 9   9       32561 non-null  object
 10  10      32561 non-null  int64 
 11  11      32561 non-null  int64 
 12  12      32561 non-null  int64 
 13  13      32561 non-null  object
 14  14      32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [24]:
max_sample = df.max()
min_sample = df.min()
steps = []

for i in range(14):
    try:
        step = int(max_sample[i]) - int(min_sample[i])
        steps.append(np.ceil(step/10))
    except:
        steps.append(-1)

steps

[8.0, -1, 147242.0, -1, 2.0, -1, -1, -1, -1, -1, 10000.0, 436.0, 10.0, -1]

In [25]:
# preprocess data set
adult_data = []

for adult in df.values:
    adult_set = set()
    for i in range(14):
        # ignore missing index
        if adult[i] == "?":
            pass
        # convert continuous data to categorical ones based on the steps.
        elif i in {0, 2, 10, 11, 12}:
            adult_set.add(df_names[i] + str(int(np.floor(adult[i]/steps[i]))))
        # ignore repeated data
        elif i == 4:
            pass
        else:
            adult_set.add(adult[i])
    adult_data.append(adult_set)

# cut the first 100 transactions as the test data set
adult_data[0]

{'Adm-clerical',
 'Bachelors',
 'Male',
 'Never-married',
 'Not-in-family',
 'State-gov',
 'United-States',
 'White',
 'age4',
 'capital-gain0',
 'capital-loss0',
 'fnlwgt0',
 'hours-per-week4'}

In [26]:
# first scan
def first_scan(data_set, min_support):
    c1 = []
    supports = []

    # generate Candidate C1 and count support
    for transaction in data_set:
        for item in transaction:
            if not {item} in c1:
                c1.append({item})
                supports.append(1)
            else:
                supports[c1.index({item})] += 1

    # compare candidates with min_support
    item_set = []
    frequent_dict = []
    for idx in range(len(c1)):
        if supports[idx] >= min_support:
            item_set.append(c1[idx])
            frequent_dict.append((c1[idx], supports[idx]))

    # generate new candidates
    temp = list(itertools.combinations(item_set, 2))
    temp = [set.union(combination[0], combination[1]) for combination in temp]
    new_item_set = []
    [new_item_set.append(candidate) for candidate in temp if not i in new_item_set]

    return frequent_dict, new_item_set

In [27]:
# a helper function use to get the subsets with a specific length from a iterable object
def power_set(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s) + 1))

def k_subsets(s, k):
    return [set(item) for item in power_set(s) if len(item) == k]

In [28]:
# general scan function
def scan(data_set, ck, min_support, print_flag=False):
    supports = [0 for candidate in ck]
    fre_data_sets = []

    # count support of these candidates
    for transaction in data_set:
        fre_flag = False
        for candidate in ck:
            if candidate.issubset(transaction):
                supports[ck.index(candidate)] += 1
                fre_flag = True

        # save the frequent transactions for the next scan, and discard the infrequent ones
        if fre_flag:
            fre_data_sets.append(transaction)

    # compare candidates with min_support
    frequent_item_sets = []
    frequent_dict= []
    for idx in range(len(ck)):
        if supports[idx] >= min_support:
            frequent_item_sets.append(ck[idx])
            frequent_dict.append((ck[idx], supports[idx]))
            if print_flag:
                print(ck[idx], supports[idx])

    if not frequent_item_sets:
        return []

    # generate new candidates
    k_num = len(frequent_item_sets[0])

    temp = list(itertools.combinations(frequent_item_sets, 2))
    temp = [combination[0].union(combination[1]) for combination in temp
            if combination[0]&combination[1] and len(combination[0]&combination[1]) == k_num-1]

    new_item_set = []

    # check whether those candidates repeated or have infrequent subsets
    for candidate in temp:
        if candidate not in new_item_set:
            subsets = k_subsets(candidate, k_num)
            flag = True
            for item in subsets:
                if item not in frequent_item_sets:
                    flag = False
                    break
            if flag:
                new_item_set.append(candidate)

    return frequent_dict, new_item_set, fre_data_sets

In [29]:
# integrate previous functions

def apriori(data_set, min_sup):
    frequent_item_set = []

    temp_frequent_dict, ck = first_scan(data_set, min_sup)
    frequent_item_set += temp_frequent_dict

    while ck:
        temp_frequent_dict, ck, data_set = scan(data_set, ck, min_sup)
        frequent_item_set += temp_frequent_dict

    return frequent_item_set

adult_fre_item_set1 = apriori(adult_data[:100], 50)

In [30]:
# fp tree node

class Node:
    def __init__(self, item, fre_count):
        self.item = item
        self.fre_count = fre_count
        self.children = set()
        self.node_link = None

    def increment(self, fre=1):
        self.fre_count += fre

    def add_child(self, new_node):
        self.children.add(new_node)
        return new_node

    def get_child(self, item):
        for child in self.children:
            if child.item == item:
                return child
        return None

    def display(self, level=0):
        print("|"*level+"-" ,self.item, self.fre_count)
        for child in self.children:
            child.display(level+1)


In [31]:
#transfer the data set to vertical version with a head table
def vertical_transfer(data_set, min_sup, frequency=None):
    if frequency:
        vertical_data = {}
        # generate the head table only with a frequency input
        for i in range(len(data_set)):
            fre_count = frequency[i]
            for item in data_set[i]:
                if not item in vertical_data.keys():
                    vertical_data[item] = fre_count
                else:
                    vertical_data[item] += fre_count

        item_seq = [[key, value, None] for key, value in vertical_data.items() if value >= min_sup]
        item_seq.sort(key=(lambda x: x[1]), reverse=True)

        return item_seq

    else:
        vertical_data = {}
        item_seq = []
        # generate the vertical data set
        for transaction in data_set:
            for item in transaction:
                if not item in vertical_data.keys():
                    vertical_data[item] = [transaction]
                    item_seq.append(item)
                else:
                    vertical_data[item].append(transaction)

        item_seq = [[item, len(vertical_data[item]), None] for item in item_seq if len(vertical_data[item]) >= min_sup]
        item_seq.sort(key=(lambda x: x[1]), reverse=True)

        return vertical_data, item_seq

In [32]:
# function construct the fp tree
def tree_construction(data_set, min_sup, display=False):
    vertical_data, head_table = vertical_transfer(data_set, min_sup)

    fp_tree = Node(None, -1)

    # scan the data_set again
    for transaction in data_set:
        current_node = fp_tree
        for item in head_table:
            if item[0] in transaction:
                child_node = current_node.get_child(item[0])
                if child_node:
                    child_node.increment()
                else:
                    child_node = current_node.add_child(Node(item[0], 1))
                    if item[2]:
                        next_link = item[2]
                        while next_link.node_link:
                            next_link = next_link.node_link
                        next_link.node_link = child_node
                    else:
                        item[2] = child_node

                current_node = child_node

    if display:
        for line in head_table:
            print(line)
        fp_tree.display()

    return head_table, fp_tree

In [33]:
# generate conditional pattern base from a fp tree
def mine_fp_tree(fp_tree: Node, curr_path, conditional_pattern_base):
    for child in fp_tree.children:
        if child.item in conditional_pattern_base.keys():
            conditional_pattern_base[child.item][0].append(curr_path.copy())
            conditional_pattern_base[child.item][1].append(child.fre_count)
        else:
            conditional_pattern_base[child.item] = ([curr_path.copy()], [child.fre_count])
        mine_fp_tree(child, curr_path+[child.item], conditional_pattern_base)

In [34]:
# generate the frequent pattern from a sub frequent tree
def mine_sub_fp_tree(fp_tree: Node, curr_path, p_sets):
    if fp_tree.children:
        for child in fp_tree.children:
            mine_sub_fp_tree(child, curr_path.union({child.item}), p_sets)
    else:
        for p_set in p_sets:
            if p_set[0].issubset(curr_path) and fp_tree.fre_count >= 0:
                p_set[1] += fp_tree.fre_count


In [35]:
def min_cp_base(conditional_pattern_base:dict, min_sup:int):
    frequent_pattern = []

    # go through each item in the conditional pattern base
    for item_name, cpb in conditional_pattern_base.items():
        sub_fp_tree = Node(None, -1)
        paths = cpb[0]
        frequencies = cpb[1]
        head_table = vertical_transfer(paths, min_sup, frequencies)

        # similar to the main fp tree construction, construct the sub fp tree
        for i in range(len(paths)):
            current_node = sub_fp_tree
            path = paths[i]
            frequency = frequencies[i]

            for item in head_table:
                if item[0] in path:
                    child_node = current_node.get_child(item[0])
                    if child_node:
                        child_node.increment(frequency)
                    else:
                        child_node = current_node.add_child(Node(item[0], frequency))
                        if item[2]:
                            next_link = item[2]
                            while next_link.node_link:
                                next_link = next_link.node_link
                            next_link.node_link = child_node
                        else:
                            item[2] = child_node

                    current_node = child_node

        #generate the frequent of each subtree
        possible_item_sets = [[set(item), 0] for item in power_set({item[0] for item in head_table}.union({item_name}))
                              if len(item) > 2 and item_name in item]

        mine_sub_fp_tree(sub_fp_tree, {item_name}, possible_item_sets)

        frequent_pattern += ([({item_name, item[0]}, item[1]) for item in head_table] +
                             [tuple(item_set) for item_set in possible_item_sets if item_set[1]>=min_sup])

    return frequent_pattern

In [36]:
def fp_growth(data_base, min_sup):
    head_table, fp_tree = tree_construction(data_base, min_sup)
    frequent_item_set = [({item[0]}, item[1]) for item in head_table]
    cp_base = {}
    mine_fp_tree(fp_tree, [], cp_base)
    frequent_item_set += min_cp_base(cp_base, min_sup)
    return frequent_item_set

adult_fre_item_set2 = fp_growth(adult_data[:100], 50)

In [37]:
# the confidence for two frequent item sets
def confidence(frequent_item0, frequent_item1, frequent_item_set):
    union = frequent_item0.union(frequent_item1)
    union_support = 0
    item0_support = 0

    for item in frequent_item_set:
        if item[0] == union:
            union_support = item[1]
        if item[0] == frequent_item0:
            item0_support = item[1]
        if union_support and item0_support:
            break
    if item0_support:
        return union_support/item0_support
    else:
        return 0

print('Apriori Male => {Male, United-States}, Confidence =',
      confidence({'Male'}, {'Male', 'United-States'} ,adult_fre_item_set1))

print('FP-Growth Male => {Male, United-States}, Confidence =',
      confidence({'Male'}, {'Male', 'United-States'} ,adult_fre_item_set2))

Apriori Male => {Male, United-States}, Confidence = 0.8513513513513513
FP-Growth Male => {Male, United-States}, Confidence = 0.8513513513513513


In [38]:
def association_generation(fre_item_set):
    for set1 in fre_item_set:
        for set2 in fre_item_set:
            if len(set2[0]) > 1 and set1[0].issubset(set2[0]) and set1[0] != set2[0]:
                print(set1[0], '=>', set2[0], ', Confidence =',
                      confidence(set1[0], set2[0] ,adult_fre_item_set1))

association_generation(adult_fre_item_set1)

{'hours-per-week4'} => {'hours-per-week4', 'capital-loss0'} , Confidence = 0.967741935483871
{'hours-per-week4'} => {'United-States', 'hours-per-week4'} , Confidence = 0.8225806451612904
{'hours-per-week4'} => {'White', 'hours-per-week4'} , Confidence = 0.8225806451612904
{'hours-per-week4'} => {'hours-per-week4', 'capital-gain0'} , Confidence = 0.9838709677419355
{'hours-per-week4'} => {'hours-per-week4', 'capital-loss0', 'capital-gain0'} , Confidence = 0.9516129032258065
{'hours-per-week4'} => {'United-States', 'hours-per-week4', 'capital-gain0'} , Confidence = 0.8064516129032258
{'hours-per-week4'} => {'White', 'hours-per-week4', 'capital-gain0'} , Confidence = 0.8064516129032258
{'capital-loss0'} => {'hours-per-week4', 'capital-loss0'} , Confidence = 0.631578947368421
{'capital-loss0'} => {'Male', 'capital-loss0'} , Confidence = 0.7473684210526316
{'capital-loss0'} => {'United-States', 'capital-loss0'} , Confidence = 0.8421052631578947
{'capital-loss0'} => {'White', 'capital-loss0'